In [2]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import cvzone
import math
import numpy as np
from sort import Sort
import ultralytics

In [ ]:
video = r"people.mp4"


cap = cv2.VideoCapture(video)
#cap.set(3, 1366)
#cap.set(4, 720)

model = YOLO("yolov8n.pt")

class_names = [
                "person", "bicycle", "car", "motorbike", "bus", "train", "truck", "boat",
                "traffic light", "fire hydrant", "stop sign", "parking metet", "bench", "bird", "cat",
                "dog", "horse", "sheep", "cow", "elephent", "bear", "aebra", "giraff", "backpacks", "umbrella",
                "habdbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball hat",
                "baseball glove", "skateboard", "surfboard", "tennis racket", "bottel", "wine glass", "cup"
                "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", "bed",
                "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                "teddy bear", "hair drier", "toothbrush"
]

people_mask = "People Counter\mask.png"
mask = cv2.imread(people_mask)
#print("mask size", mask.shape)
#mask = cv2.resize(mask, (1366, 720))
#print("mask", mask.shape)



tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits_up = [103, 161, 296, 161] 
limits_down = [527, 489, 735, 489] 

total_count_up = []
total_count_down = []

while True:
    success, img = cap.read() 
    #print(img.shape)
    #img = cv2.resize(img, (720, 1280))
    img_region = cv2.bitwise_and(img, mask)

    graphics_photo = "graphics.png"
    graphics_img = cv2.imread(graphics_photo, cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, graphics_img, (730, 260))
    
    #results = model(img, stream=True)
    results = model(img_region, stream=True)

    detections = np.empty((0, 5))
    for r in results:
        boxes = r.boxes
        for box in boxes:

            #Bounding box
            x, y, x1, y1 = box.xyxy[0]
            x, y, x1, y1 = int(x), int(y), int(x1), int(y1)
            #print(x, y, x1, y1)
            #cv2.rectangle(img, (x, y), (w, h), (0, 255, 0), 5)

            w, h = x1 - x, y1 - y
            bbox = int(x), int(y), int(w), int(h)
            #cvzone.cornerRect(img, (x, y, w, h), l=9)

            #Congidence 
            conf = math.ceil((box.conf[0] * 100)) / 100
            #print(conf)
            #cvzone.putTextRect(img, f"{conf}", (max(0, x1), max(35, y1)))

            #Class name
            cls = int(box.cls[0])

            current_class = class_names[cls]

            if current_class == "person" and conf > 0.3:
                #cvzone.putTextRect(img, f"{class_names[cls]} {conf}", (max(0, x), max(35, y)), scale=0.6, thickness=1,
                                  #offset=3)
                #cvzone.cornerRect(img, (x, y, w, h), l=9, rt=5)
                current_array = np.array([x, y, x1, y1, conf])
                detections = np.vstack((detections, current_array))

    results_for_tracker = tracker.update(detections)
    
    cv2.line(img, (limits_up[0], limits_up[1]), (limits_up[2], limits_up[3]), (0, 0, 255), 5)
    
    cv2.line(img, (limits_down[0], limits_down[1]), (limits_down[2], limits_down[3]), (0, 0, 255), 5)

    for result in results_for_tracker:
        x, y, x1, y1, id = result
        x, y, x1, y1, id = int(x), int(y), int(x1), int(y1), int(id)

        print(result)
        
        w, h = x1 - x, y1 - y
        cvzone.cornerRect(img, (x, y, w, h), l=9, rt=2, colorR=(255, 0, 0))
        cvzone.putTextRect(img, f"{id}", (max(0, x), max(35, y)), scale=2, thickness=3,
                                  offset=10)
        cx, cy = x + w //2, y + h //2
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

        if limits_up[0] < cx < limits_up[2] and limits_up[1] - 15 < cy < limits_up[1] + 15:
            if total_count_up.count(id) == 0:
                total_count_up.append(id)
                cv2.line(img, (limits_up[0], limits_up[1]), (limits_up[2], limits_up[3]), (0, 255, 0), 5)

    if limits_down[0] < cx < limits_down[2] and limits_down[1] - 15 < cy < limits_down[1] + 15:
            if total_count_down.count(id) == 0:
                total_count_down.append(id)
                cv2.line(img, (limits_down[0], limits_down[1]), (limits_down[2], limits_down[3]), (0, 255, 0), 5)

    #cvzone.putTextRect(img, f" Count : {len(total_count_up)}", (50, 50))
    cv2.putText(img, str(len(total_count_up)), (929, 345), cv2.FONT_HERSHEY_PLAIN, 5, (139, 195, 75), 7)
    
    cv2.putText(img, str(len(total_count_down)), (1185, 345), cv2.FONT_HERSHEY_PLAIN, 5, (50, 50, 230), 7)

    cv2.imshow("image", img)
    #cv2.imshow("Region", img_region)
    cv2.waitKey(1) 